# How to convert Runnables to Tools

# Step 1: Define the Runnable
A Runnable is any callable entity that processes input and produces output. For instance, a simple function that multiplies numbers can be wrapped as a Runnable.

In [1]:
from langchain_core.runnables import RunnableLambda

# Define a simple callable
def multiply(args: dict) -> str:
    return str(args["a"] * max(args["b"]))

# Wrap it as a Runnable
runnable = RunnableLambda(multiply)


# Step 2: Convert to a Tool
A Tool extends Runnable with additional constraints such as input validation and metadata like name and description.

 You can convert a Runnable into a Tool using the .as_tool() method.

Example with Typed Input

If your function has well-defined input types, you can specify them using TypedDict:

In [2]:
from typing import List
from typing_extensions import TypedDict

# Define input schema
class Args(TypedDict):
    a: int
    b: List[int]

# Wrap the Runnable and convert to a Tool
runnable_tool = runnable.as_tool(
    name="Multiplier Tool",
    description="Multiplies 'a' with the maximum of the list 'b'.",
)


C:\Users\Admin\AppData\Local\Temp\ipykernel_16532\1632586118.py:10: LangChainBetaWarning: This API is in beta and may change in the future.
  runnable_tool = runnable.as_tool(


Example with args_schema

Alternatively, you can define a custom schema for the tool arguments using pydantic:

In [3]:
from pydantic import BaseModel, Field

# Define a schema
class MultiplySchema(BaseModel):
    a: int = Field(..., description="The base integer.")
    b: List[int] = Field(..., description="List of integers.")

# Convert to a Tool with schema
runnable_tool = runnable.as_tool(args_schema=MultiplySchema)


# Step 3: Use the Tool
Once your Runnable is converted to a Tool, you can invoke it like any other tool in LangChain:


In [4]:
# Invoking the tool
result = runnable_tool.invoke({"a": 3, "b": [1, 2, 5]})
print(result)  # Output: "15"


15


# Step 4: Use Tools in Agents
Tools can be passed to agents for interactive applications. For example, here’s how to integrate the tool with a LangGraph agent:

Set up a Chat Model

In [ ]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(model="gpt-4o-mini")


Integrate Tool with Agent

In [7]:
from langgraph.prebuilt import create_react_agent

# Add your tool to the agent
agent = create_react_agent(llm, tools=[runnable_tool])

# Query the agent
response = agent.invoke({"messages": [("human", "Multiply 3 with the max of [1, 2, 5].")]})
print(response)


{'messages': [HumanMessage(content='Multiply 3 with the max of [1, 2, 5].', additional_kwargs={}, response_metadata={}, id='c80cec9c-49c6-4156-8f58-7934a4eac76c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xhQevIVmQGPJSJAxfPDlarFR', 'function': {'arguments': '{"a":3,"b":[1,2,5]}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 148, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-70c758a5-fb34-4185-b4ec-ec680095d4b7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': [1, 2, 5]}, 'id': 'call_xhQevIVmQGPJSJAxfPDlarFR', 'type': 'tool_call'}], usage_metadata